# Libraries

In [1]:
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score
)
from optuna.visualization import (
    plot_optimization_history, 
    plot_param_importances, 
    plot_slice
)
from sklearn import set_config

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import optuna
import shap

# Configuration

In [2]:
# Global configurations for sklearn:
set_config(transform_output="pandas")

# Global configurations for pandas:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 50)
pd.set_option("display.precision", 3)
pd.set_option("display.max_colwidth", None)

In [3]:
SEED = 42

DATA_DIR = 'data/'

# Functions

In [4]:
def test_func():
    pass

# Read data

In [7]:
# Original data
org_data = pd.read_csv(DATA_DIR + 'flood.csv')

# Competition data
org_train = pd.read_csv(DATA_DIR + 'train.csv')

In [8]:
# Enrich dataset
org_train.drop('id', axis=1, inplace=True)
df_master = pd.concat([org_train, org_data])

# Leave the original data as is
df = df_master.copy()
df.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,IneffectiveDisasterPreparedness,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,5,8,5,8,6,4,4,3,3,4,2,5,3,3,5,4,7,5,7,3,0.445
1,6,7,4,4,8,8,3,5,4,6,9,7,2,0,3,5,3,3,4,3,0.450
2,6,5,6,7,3,7,1,5,4,5,6,7,3,7,5,6,8,2,3,3,0.530
3,3,4,6,5,4,8,4,7,6,8,5,2,4,7,4,4,6,5,7,5,0.535
4,5,3,2,6,4,4,3,3,3,3,5,2,2,6,6,4,1,2,3,5,0.415


# EDA

# Pre-processing

# Feature engineering

# Metric

# Modeling

# Evaluation

# Prediction & Submission

# Reference